<a href="https://colab.research.google.com/github/alrb2005/MechaineLearing-0716-/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 5s 0us/step




---


從 tf.keras 那邊取得資料並解壓縮


---



In [ ]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [ ]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df



---


用os.path.dirname 去看dataset的路徑

使用glob.glob去看資料夾內的東西

因為train跟test都是路徑中間的名子，所以在設定函式時，讓它是mid


---



In [ ]:
train_df = getdata("train")
test_df = getdata("test")

train_df #記得檢查資料

,content,sentiment
0,"""Protocol"" is a hit-and-miss picture starring ...",1
1,The third Fred MacMurray/Carole Lombard film i...,1
2,"Like his earlier film, ""In a Glass Cage"", Agus...",1
3,"don't mind the soundtrack, which is played out...",1
4,Virgil Manoven is an old man who lives alone i...,1
...,...,...
24995,Kirsten Dunst is terribly overrated as an actr...,0
24996,Gymkata is without a doubt one of the worst mo...,0
24997,This is by far the worst movie I've ever seen....,0
24998,"An uninspired and undistinguished ""new"" Columb...",0


In [ ]:
#Tokenizer:把詞化成數字
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000) #設定只有3000個字是認識的，所以讓它篩3000個字是有用的
tok.fit_on_texts(train_df["content"])


In [ ]:
#這裡面幫你按順序排列，出現次數多到少
tok.word_index #裡面沒有"0",因為0是拿來padding


{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'he': 26,
 'be': 27,
 'one': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'who': 34,
 'so': 35,
 'from': 36,
 'like': 37,
 'her': 38,
 'or': 39,
 'just': 40,
 'about': 41,
 "it's": 42,
 'out': 43,
 'if': 44,
 'has': 45,
 'some': 46,
 'there': 47,
 'what': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'she': 56,
 'even': 57,
 'my': 58,
 'would': 59,
 'which': 60,
 'only': 61,
 'story': 62,
 'really': 63,
 'see': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'were': 68,
 'me': 69,
 'well': 70,
 'than': 71,
 'we': 72,
 'much': 73,
 'been': 74,
 'bad': 75,
 'get': 76,
 'will': 77,
 'do': 78,
 'also': 79,
 'into': 80,
 'people': 81,
 'other': 82,
 '



---


一篇文章需要截長補短(讓長度一致)
因為要讓每個文章embadding時的情緒量一樣多
(假設這次設定情緒128個，情緒量=128*文字數量)

截長:256/512個字(長度)

補短:補0補成一樣長度


---



In [ ]:
# sequences: 數字序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)
#把文章攤開來排

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,6,3,566,2,714,428,1180,14,3.0,34.0,28.0,311.0,1.0,110.0,4.0,3.0,36.0,32.0,586.0,1.0,252.0,1237.0,457.0,3.0,2.0,211.0,3.0,159.0,289.0,777.0,15.0,1.0,1203.0,587.0,1365.0,77.0,1.0,4.0,667.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,837,1794,19,6,3,224,50,618.0,71.0,953.0,635.0,1.0,2696.0,2.0,1.0,2537.0,263.0,635.0,42.0,243.0,157.0,4.0,1.0,294.0,60.0,681.0,13.0,1059.0,142.0,8.0,1.0,248.0,7.0,7.0,1.0,201.0,294.0,2161.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37,24,904,19,8,3,1927,32,1588.0,2.0,542.0,12.0,6.0,57.0,50.0,2006.0,71.0,91.0,11.0,6.0,28.0,4.0,1.0,88.0,2006.0,2.0,2289.0,1264.0,28.0,97.0,123.0,64.0,1.0,450.0,2.0,4.0,564.0,20.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89,327,1,812,60,6,253,43,31.0,147.0,7.0,7.0,128.0,6.0,207.0,218.0,2.0,134.0,11.0,13.0,32.0,904.0,214.0,15.0,38.0,56.0,6.0,176.0,49.0,14.0,247.0,36.0,1.0,352.0,496.0,4.0,1.0,1448.0,8.0,1467.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,32,151,129,34,453,581,8,24.0,2932.0,24.0,2753.0,1127.0,28.0,1965.0,80.0,1.0,1408.0,184.0,24.0,48.0,269.0,37.0,1.0,585.0,4.0,3.0,182.0,503.0,8.0,1.0,652.0,4.0,1.0,1408.0,26.0,1.0,819.0,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,6,1899,14,32,521,22,67,373,207.0,438.0,40.0,113.0,10.0,37.0,148.0,111.0,6.0,355.0,1.0,527.0,45.0,54.0,144.0,5.0,1.0,102.0,112.0,1272.0,34.0,5.0,63.0,16.0,39.0,57.0,456.0,41.0,547.0,750.0,17.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,6,206,3,822,28,4,1,246,99.0,123.0,90.0,18.0,21.0,1.0,75.0,240.0,4.0,75.0,99.0,11.0,28.0,6.0,35.0,370.0,42.0,250.0,5.0,103.0,2.0,459.0,2097.0,692.0,124.0,21.0,25.0,3.0,608.0,8.0,113.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,6,31,227,1,246,17,204,123.0,107.0,36.0,1.0,111.0,148.0,1.0,662.0,1.0,315.0,299.0,1.0,113.0,2.0,119.0,10.0,1044.0,1.0,111.0,172.0,682.0,152.0,8.0,9.0,2069.0,1994.0,7.0,7.0,11.0,6.0,3.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,32,2,159,2533,60,1082,1,129,8.0,1.0,24.0,1758.0,61.0,15.0,24.0,5.0,20.0,65.0,1758.0,311.0,2533.0,7.0,7.0,2.0,12.0,9.0,6.0,41.0,9.0,2.0,1235.0,16.0,3.0,1032.0,226.0,345.0,12.0,9.0,6.0,395.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1365,77,1,4,667,8,260,123,27,1,169,6,318,14,641,57,148,215,14,160,14,38,115,19,1951,18,42,128,3,49,17,2,10,119,459,7,7,43,4,685
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,79,2043,6,8,28,4,24,105,14,3,20,1,94,15,7,7,309,361,1773,53,144,324,636,2,10,583,27,767,44,72,64,243,157,307,4,15,1,1780,83,1114
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,12,93,9,2640,122,32,217,1164,5,27,35,117,1,347,2,243,35,2142,5,27,35,2,243,35,2,57,1,23,4,1,741,1484,2,542,11,19,6,328,894
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2321,311,134,9,66,91,385,1,2103,23,721,2798,30,11,210,7,7,287,316,15,56,6,128,35,1449,2,112,66,12,359,72,23,5,31,96,108,1504,636,690,155
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,217,337,4,411,46,136,23,2064,2,1,113,6,318,1,1618,726,1751,955,4,1,1408,2997,145,340,8,1,2974,1714,199,11,676,186,19,3,576,785,43,4,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,52,160,550,111,6,21,1726,39,2568,8,1,219,22,67,64,254,111,468,579,3,242,48,6,11,17,421,5,27,209,450,34,2256,22,525,31,1,127,4,11,19
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,8,547,1266,10,89,101,9,13,976,5,27,11,75,18,34,67,373,42,21,495,18,44,22,178,139,2,12,77,94,3,49,2597,848,163,15,32,2190,4,250
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,1,61,279,10,267,140,11,433,592,6,5,166,43,86,1,607,67,33,1364,11,17,7,7,89,103,11,44,22,1104,55,123,1551,232,23,3,434,8,11,418
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,30,1,1758,60,1470,1459,34,13,47,7,7,4,172,2533,611,10,194,10,13,146,32,387,4,2192,887,32,2381,5,1,476,4,1,198,16,1844,1419,402,113,52,535




---


keras.layers.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)


---


input_dim = 我們設定3000，所以有3000dim，但不能忘記0，所以我們有3001個dim

mask_zero = 告訴它，裡面的0是我設定padding用的，所以要設定True


---



In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
INPUT_DIM = 3001
OUTPUT_DIM = 128
INPUT_LENGTH = 512
layers = [
    Embedding(INPUT_DIM, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               16777472  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 17,162,114
Trainable params: 17,162,114
Non-trainable params: 0
_________________________________________________________________




---

截取 MLP 過來使用


---




In [ ]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("embedding.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)       
]
model.fit(x_train_pad,
     y_train, 
     batch_size=200,
     validation_split=0.1,
     epochs=50,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 61s - loss: 0.7384 - accuracy: 0.5621 - val_loss: 0.8591 - val_accuracy: 0.1656
Epoch 2/50
113/113 - 61s - loss: 0.3935 - accuracy: 0.8236 - val_loss: 0.3538 - val_accuracy: 0.8640
Epoch 3/50
113/113 - 61s - loss: 0.2048 - accuracy: 0.9234 - val_loss: 0.2676 - val_accuracy: 0.8940
Epoch 4/50
113/113 - 61s - loss: 0.1043 - accuracy: 0.9673 - val_loss: 0.4051 - val_accuracy: 0.8620
Epoch 5/50
113/113 - 60s - loss: 0.0359 - accuracy: 0.9914 - val_loss: 0.5102 - val_accuracy: 0.8480
Epoch 6/50
113/113 - 61s - loss: 0.0116 - accuracy: 0.9988 - val_loss: 0.6195 - val_accuracy: 0.8428
Epoch 7/50
113/113 - 63s - loss: 0.0047 - accuracy: 0.9998 - val_loss: 0.6442 - val_accuracy: 0.8488
Epoch 8/50
113/113 - 61s - loss: 0.0028 - accuracy: 0.9998 - val_loss: 0.8867 - val_accuracy: 0.8152


In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 22s 28ms/step - loss: 0.3050 - accuracy: 0.8727


[0.3049832880496979, 0.8727200031280518]

In [ ]:
# get_weights/set_weights
embedding = [
    Embedding(INPUT_DIM, OUTPUT_DIM, mask_zero=True)
]
w = model.layers[0].get_weights()
partial = Sequential(embedding)
partial.layers[0].set_weights(w)
partial.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         384128    
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
result = partial.predict([tok.word_index["bill"]])
print("bill:", result[0][0])

bill: [-0.03828952  0.03326032  0.02427153  0.04679516  0.04207686 -0.02190334
  0.00630224 -0.00725147 -0.05559356 -0.02578219 -0.03089755  0.09110145
  0.04655362  0.06261261 -0.0357485   0.05148795 -0.00719833 -0.03957763
  0.04340322  0.03528786  0.04397859  0.03639803  0.00592729 -0.02746095
  0.00693097  0.00537604  0.04879481 -0.02067654 -0.02042959 -0.00412153
 -0.01154345  0.04871573 -0.05479597 -0.03720953  0.04639733 -0.0024646
  0.06487384 -0.02241205  0.03927561 -0.0753419  -0.03145753 -0.00663924
  0.04339033 -0.06149572 -0.00446341 -0.03603504 -0.01379819  0.02711402
 -0.01660223 -0.07784478 -0.01655978  0.05819701  0.06096318 -0.03755122
 -0.00993491  0.07593333  0.06375194 -0.06270331 -0.03259164 -0.05100629
  0.04474508 -0.05962748  0.02525196  0.02961193 -0.01341406  0.04807772
  0.06263497 -0.02082695 -0.01685778 -0.0183027  -0.00619077 -0.0492624
 -0.02016917  0.0030521   0.00747305 -0.06764244 -0.05421399 -0.02119561
 -0.05084822  0.04881195  0.01725728  0.0286404